# Introduction
This is a colab notebook. It is a special case of a ipython notebook in a specific interface. This is how you will do your analysis for this class. It requires a google account and is an example of an ipython notebook interface. 

This is an example of a text block. To add text blocks, simply click the "+ Text" button on the bar above the notebook. 

The code is marked up in a markup language called "Markdown". You can find a guide to markdown syntax here: [https://www.markdownguide.org/cheat-sheet/](https://www.markdownguide.org/cheat-sheet/).

Also, the [colab documentation](https://colab.research.google.com/notebooks/markdown_guide.ipynb) is useful.

Markdown should take you only a few moments to learn. It is as easy as a markup langauge can be made.



# Imports and files
The first thing we'll try is loading some data and plotting it. To do this, we'll need some packages. Let's load up pandas, a package for data management, and matplotlib. The python command for this is `import`.


In [0]:
import pandas as pd
import matplotlib as mpl

If you want to mount your google drive, do this. 

```
from google.colab import drive
drive.mount('/content/drive/')
```

We won't do this here, since we'll download our data from github.

To execute unix commands, do `!command`. For example, here I'm listing out the contents of the connected runtime for this notebook. For example, here I'm running the `ls` command for listing out the contents of a unix directory.

In [38]:
!ls

kirby127a_3_1_ax_283Labels_M2_corrected_stats.csv    sample_data
kirby127a_3_1_ax_283Labels_M2_corrected_stats.csv.1


Let's download the data. Or you could upload the data to google drive or directly upload to colab. 

In [39]:
!wget https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/kirby127a_3_1_ax_283Labels_M2_corrected_stats.csv

--2019-08-02 19:07:18--  https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/kirby127a_3_1_ax_283Labels_M2_corrected_stats.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61456 (60K) [text/plain]
Saving to: ‘kirby127a_3_1_ax_283Labels_M2_corrected_stats.csv.2’

kirby127a_3_1_ax_28 100%[===================>]  60.02K  --.-KB/s    in 0.01s   

2019-08-02 19:07:19 (4.13 MB/s) - ‘kirby127a_3_1_ax_283Labels_M2_corrected_stats.csv.2’ saved [61456/61456]



# Reading data in with pandas

Let's now read in an MRICloud dataset using pandas. We want to use the function `read_csv` within pandas. Notice we imported pandas `as pd` so the command is `pd.read_csv`. 

In [0]:
df = pd.read_csv("/content/kirby127a_3_1_ax_283Labels_M2_corrected_stats.csv")

Let's look at the first 4 rows of our dataframe. The object `dataset` is a `pandas` object with associated methods. One is `head` which allows one to see the first few rows of data. 

In [41]:
df.head(4)

,Unnamed: 0,rawid,roi,volume,min,max,mean,std,type,level
0,1,kirby127a_3_1_ax.img,Telencephalon_L,531111,0,374,128.3013,51.8593,1,1
1,2,kirby127a_3_1_ax.img,Telencephalon_R,543404,0,300,135.0683,53.6471,1,1
2,3,kirby127a_3_1_ax.img,Diencephalon_L,9683,15,295,193.5488,32.2733,1,1
3,4,kirby127a_3_1_ax.img,Diencephalon_R,9678,10,335,193.7051,32.7869,1,1


# Working with the data

Let's get rid of the column `rawid` and the unnamed column since they're kind of useless for today's lecture. Also let's work with only the volume.

In [0]:
df = df.drop(['Unnamed: 0', 'rawid', 'min', 'max', 'mean', 'std'], axis = 1)

Now let's create a column called `icv` for intra-cranial volume. ICV is defined as the summ of the Type I Level 1 structures and cerebrospinal fluid. For the rest of this lecture, we're just going to look at this type and level.

In [43]:
## Extract the Type 1 Level 1 data
t1l1 = df.loc[(df['type'] == 1) & (df['level'] == 1)]
## Create a new column based on ICV
t1l1 = t1l1.assign(icv = sum(t1l1['volume']))
t1l1

,roi,volume,type,level,icv
0,Telencephalon_L,531111,1,1,1378295
1,Telencephalon_R,543404,1,1,1378295
2,Diencephalon_L,9683,1,1,1378295
3,Diencephalon_R,9678,1,1,1378295
4,Mesencephalon,10268,1,1,1378295
5,Metencephalon,159402,1,1,1378295
6,Myelencephalon,4973,1,1,1378295
7,CSF,109776,1,1,1378295


Now the TBV is defined as the sum of all columns except CSF. 

In [44]:
t1l1 = t1l1.assign(tbv = sum(t1l1['volume'][(t1l1['roi'] != 'CSF')]))
t1l1

,roi,volume,type,level,icv,tbv
0,Telencephalon_L,531111,1,1,1378295,1268519
1,Telencephalon_R,543404,1,1,1378295,1268519
2,Diencephalon_L,9683,1,1,1378295,1268519
3,Diencephalon_R,9678,1,1,1378295,1268519
4,Mesencephalon,10268,1,1,1378295,1268519
5,Metencephalon,159402,1,1,1378295,1268519
6,Myelencephalon,4973,1,1,1378295,1268519
7,CSF,109776,1,1,1378295,1268519


Let's look at brain composition.

In [48]:
t1l1.assign(comp = lambda x: x.volume / x.tbv)

,roi,volume,type,level,icv,tbv,comp
0,Telencephalon_L,531111,1,1,1378295,1268519,0.418686
1,Telencephalon_R,543404,1,1,1378295,1268519,0.428377
2,Diencephalon_L,9683,1,1,1378295,1268519,0.007633
3,Diencephalon_R,9678,1,1,1378295,1268519,0.007629
4,Mesencephalon,10268,1,1,1378295,1268519,0.008094
5,Metencephalon,159402,1,1,1378295,1268519,0.125660
6,Myelencephalon,4973,1,1,1378295,1268519,0.003920
7,CSF,109776,1,1,1378295,1268519,0.086539
